In [1]:
from peewee import *
import base64

db = SqliteDatabase('koth_training.db')

class Training_data(Model):
  id = IntegerField()
  binary = BlobField()
  evals = FloatField()

  class Meta:
    database = db

  def binary_base64(self):
    return base64.b64encode(self.binary)
db.connect()
LABEL_COUNT = 1000000
print(LABEL_COUNT)
eval = Training_data.get(Training_data.id == 1)
print(eval.binary_base64())

1000000
b'APcACAAAAAAAAAAACAD3AEIAAAAAAAAAAAAAAAAAAEIkAAAAAAAAAAAAAAAAAAAkgQAAAAAAAAAAAAAAAAAAgRAAAAAAAAAAAAAAAAAAABAIAAAAAAAAAAAAAAAAAAAI+AAAAFM='


In [2]:
import os
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset, random_split
import pytorch_lightning as pl
from random import randrange

class EvaluationDataset(IterableDataset):
  def __init__(self, count):
    self.count = count
  def __iter__(self):
    return self
  def __next__(self):
    idx = randrange(self.count)
    return self[idx]
  def __len__(self):
    return self.count
  def __getitem__(self, idx):
    eval = Training_data.get(Training_data.id == idx+1)
    bin = np.frombuffer(eval.binary, dtype=np.uint8)
    bin = np.unpackbits(bin, axis=0).astype(np.single) 
    eval.evals = max(eval.evals, -15)
    eval.evals = min(eval.evals, 15)
    ev = np.array([eval.evals]).astype(np.single) 
    return {'binary':bin, 'eval':ev}    

dataset = EvaluationDataset(count=LABEL_COUNT)

In [9]:
import time
from collections import OrderedDict

class EvaluationModel(pl.LightningModule):
  def __init__(self,learning_rate=1e-3,batch_size=1024,layer_count=10):
    super().__init__()
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    layers = []
    for i in range(layer_count-1):
      layers.append((f"linear-{i}", nn.Linear(808, 808)))
      layers.append((f"relu-{i}", nn.ReLU()))
    layers.append((f"linear-{layer_count-1}", nn.Linear(808, 1)))
    self.seq = nn.Sequential(OrderedDict(layers))

  def forward(self, x):
    return self.seq(x)

  def training_step(self, batch, batch_idx):
    x, y = batch['binary'], batch['eval']
    y_hat = self(x)
    loss = F.l1_loss(y_hat, y)
    self.log("train_loss", loss)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

  def train_dataloader(self):
    dataset = EvaluationDataset(count=LABEL_COUNT)
    return DataLoader(dataset, batch_size=self.batch_size, num_workers=0)

configs = [
           #{"layer_count": 4, "batch_size": 512},
           {"layer_count": 6, "batch_size": 1024},
           ]
for config in configs:
  version_name = f'{int(time.time())}-batch_size-{config["batch_size"]}-layer_count-{config["layer_count"]}'
  logger = pl.loggers.TensorBoardLogger("lightning_logs", name="chessml", version=version_name)
  trainer = pl.Trainer(gpus=1,precision=16,max_epochs=1,auto_lr_find=True,logger=logger)
  model = EvaluationModel(layer_count=config["layer_count"],batch_size=config["batch_size"],learning_rate=1e-3)
  #trainer.tune(model)
  #lr_finder = trainer.tuner.lr_find(model, min_lr=1e-6, max_lr=1e-3, num_training=25)
  #fig = lr_finder.plot(suggest=True)
  #fig.show()
  trainer.fit(model)
  break

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | seq  | Sequential | 3.3 M 
------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
6.538     Total estimated model params size (MB)


Epoch 0:   6%|▌         | 118/1954 [04:46<1:14:17,  2.43s/it, loss=inf, v_num=nt-4]

Finding best initial lr: 100%|██████████| 25/25 [04:05<00:00,  9.84s/it]


Epoch 0:  53%|█████▎    | 513/977 [03:28<03:08,  2.46it/s, loss=2.54, v_num=nt-6]

Epoch 0: 100%|██████████| 977/977 [06:37<00:00,  2.46it/s, loss=2.22, v_num=nt-6]


In [10]:
torch.save(model.state_dict(), 'model_2-22.pt')

In [51]:
model = EvaluationModel(layer_count=4,batch_size=1024,learning_rate=1e-3)
model.load_state_dict(torch.load('model_2-3.pt'))
model.eval()
start = time.time_ns()
bin = np.frombuffer(eval.binary, dtype=np.uint8)
bin = np.unpackbits(bin, axis=0).astype(np.single) 
bin = torch.from_numpy(bin)
x = model(bin)
end = time.time_ns()
print(x, end-start)

tensor([0.7787], grad_fn=<AddBackward0>) 0
